<a href="https://colab.research.google.com/github/EryginYaroslav/AlgoSirius2024/blob/main/task_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import numpy as np

class HungarianAlgorithm:
    def __init__(self, cost_matrix):
        self.original_cost_matrix = np.array(cost_matrix)  # Оригинальная матрица стоимости.
        self.cost_matrix = np.array(cost_matrix)
        self.n = self.cost_matrix.shape[0]  # Размер матрицы.
        self.row_covered = np.zeros(self.n, dtype=bool)
        self.col_covered = np.zeros(self.n, dtype=bool)
        self.marked = np.zeros((self.n, self.n), dtype=int)
        self.path = []

    def step1(self):
        for i in range(self.n):
            self.cost_matrix[i] -= np.min(self.cost_matrix[i])

    def step2(self):
        for i in range(self.n):
            for j in range(self.n):
                if self.cost_matrix[i, j] == 0 and not self.row_covered[i] and not self.col_covered[j]:
                    self.marked[i, j] = 1
                    self.row_covered[i] = True
                    self.col_covered[j] = True
        self.row_covered[:] = False
        self.col_covered[:] = False

    def step3(self):
        for i in range(self.n):
            for j in range(self.n):
                if self.marked[i, j] == 1:
                    self.col_covered[j] = True
        return np.sum(self.col_covered) == self.n

    def step4(self):
        while True:
            row, col = self.find_a_zero()
            if row is None:
                return
            self.marked[row, col] = 2
            star_col = np.where(self.marked[row] == 1)[0]
            if len(star_col) == 0:
                self.path.append((row, col))
                self.step5()
                return
            else:
                self.row_covered[row] = True
                self.col_covered[star_col[0]] = False

    def step5(self):
        while True:
            row, col = self.path[-1]
            star_row = np.where(self.marked[:, col] == 1)[0]
            if len(star_row) > 0:
                self.path.append((star_row[0], col))
            else:
                break
            star_col = np.where(self.marked[row] == 2)[0]
            self.path.append((row, star_col[0]))

        for r, c in self.path:
            self.marked[r, c] = 1 if self.marked[r, c] != 1 else 0

        self.row_covered[:] = False
        self.col_covered[:] = False
        self.path = []

    def find_a_zero(self):
        for i in range(self.n):
            for j in range(self.n):
                if self.cost_matrix[i, j] == 0 and not self.row_covered[i] and not self.col_covered[j]:
                    return i, j
        return None, None

    def step6(self):
        min_val = np.min(self.cost_matrix[~self.row_covered][:, ~self.col_covered])
        for i in range(self.n):
            if self.row_covered[i]:
                self.cost_matrix[i] += min_val
        for j in range(self.n):
            if not self.col_covered[j]:
                self.cost_matrix[:, j] -= min_val

    def solve(self):
        self.step1()
        self.step2()
        while not self.step3():
            self.step4()
            self.step6()
        # Оптимальное сопоставление
        result = [(i, np.where(self.marked[i] == 1)[0][0]) for i in range(self.n)]
        # Рассчитать минимальную стоимость по оригинальной матрице.
        total_cost = sum(self.original_cost_matrix[i, j] for i, j in result)
        return result, total_cost


# Пример использования
if __name__ == "__main__":
    # Матрица стоимости
    cost_matrix = [
        [4, 1, 3],
        [2, 0, 5],
        [3, 2, 2]
    ]

    # Решение
    hungarian = HungarianAlgorithm(cost_matrix)
    result, total_cost = hungarian.solve()

    print("Оптимальное сопоставление (строка -> столбец):")
    for row, col in result:
        print(f"{row} -> {col}")
    print(f"Минимальная стоимость: {total_cost}")


Оптимальное сопоставление (строка -> столбец):
0 -> 1
1 -> 0
2 -> 2
Минимальная стоимость: 5
